In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%%script echo
#%%writefile "../openaigpt.py"
import os, sys

sys.path.append(os.path.expanduser("~/.django") )
import my_config
from my_config import *

from openai import OpenAI

client = OpenAI(
  organization='org-IuMjyWnHvrzUr9jsi0dT6QmL',
  project='proj_5AGDENKBvMR6sXoxezxXaWUY',
  api_key=my_config.OPENAPI_GEO_KEY
)

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    # Write your prompt
    prompt="____",
    max_tokens=200
)

print(response.choices[0].text)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
  ]
)

print(completion.choices[0].message)

from openai import OpenAI
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "write a haiku about ai"}
    ]

# OLLAMA

In [27]:
#%%writefile "../olgpt.py"
import ollama, torch, logging,datetime
from ollama import Client
from mangorest.mango import webapi
sys.path.append(os.path.expanduser("~/.django") )
    
logger = logging.getLogger( "geoapp" )
device = "cpu"
if (torch.cuda.is_available() ):
    device = "cuda"


OLLAMA_HOST= 'http://10.0.0.29:11434'
OLLAMA_HOST= 'http://67.162.141.146:11434'
OLLAMA_HOST=None
if (os.path.exists(os.path.expanduser("~/.django/my_config.py"))):
    import my_config
    from my_config import OLLAMA_HOST

OLLAMA = Client(host=OLLAMA_HOST)
#OLLAMA = Client()
#--------------------------------------------------------------------------------------------------------    
@webapi("/ollama/generate/")
def ollma_generate(request=None, model="mistral", prompt="", stream=True,**kwargs):
    logger.info(f"Prepping: {datetime.datetime.now()}")
    response = OLLAMA.generate(model=model, prompt=prompt, stream=stream)
    # Stream response
    ret = ""
    for chunk in response:
        data = chunk["response"]
        ret += data
        
    logger.debug(ret)
    return ret

ollma_generate(prompt="whos is the chief data scientist of  Martian", stream=True)   


2024-08-11 22:57:27,155 geoapp INFO: Prepping: 2024-08-11 22:57:27.155247
2024-08-11 22:57:31,136 httpx INFO: HTTP Request: POST http://10.0.0.29:11434/api/generate "HTTP/1.1 200 OK"


' I apologize for any confusion, but there seems to be a misunderstanding in your question. As of now, there isn\'t a publicly known Chief Data Scientist for "Martian." Martian could refer to various things, such as a company, a research organization, or even Mars the planet. To provide an accurate answer, more context is required.'

# Experiments